In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Permute, Dropout, Reshape, LSTM, BatchNormalization,TimeDistributed
from keras.layers import Conv2D
from keras.optimizers import Adam #, RMSprop, SGD
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from time import time

import pickle
import random
import numpy as np
import sys
import h5py


Using TensorFlow backend.


In [2]:
#scenes
batch_size =  64
nb_classes = 42
# input dimensions  
rows, cols , channels = 1292, 128,2    
#model params
nb_epoch = 50
pool_size = (3,3)                  # size of pooling area for max pooling
prob_drop_conv = 0.25                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer
fold=1
%run shared_functions.ipynb


In [3]:
# 2D data - two channel input!

#Load yogi
#import random 

def getLabelMat(scene):
    with open(scene, "rb" ) as scenesample:
        labelMat = pickle.load(scenesample)
    return labelMat

def getStackFeatMat(scene):
    with open(scene, "rb" ) as scenesample:
        l,fv = pickle.load(scenesample)
    return fv,l

def numerise(labmat, nb_classes):
    r,c=labmat.shape
    nummat=np.zeros((labmat.shape))
    for i in range(0,r):
        for j in range(0,c):
            if labmat[i,j]==1:
                nummat[i,j]=i
    return np_utils.to_categorical(nummat, nb_classes)

def reduceLabels(labels):
    r,c=labels.shape  #(should be 1292,42)
    npOfLabels=np.zeros(c)
    for j in range(0,c):
        if np.sum(labels[:,j])>0:
            npOfLabels[j]=1
    print(npOfLabels.shape)
    return npOfLabels

def getTrainDataForFold(foldNo):
    n_bg,bg_labels=countBackgrounds('inputsounds\\background\\train')
    src_files='stacked_for_classification\\train\\'+str(foldNo)+'\\*'
    validationIDs = random.sample(range(1,9), 1)
    print(validationIDs)
    vf_list=[]
    tf_list=[]
    vl_list=[]
    tl_list=[]
    labels={}
    
    while len(vf_list)==0:
        for bgl in bg_labels.keys():
            s_files=src_files+bgl+'_*2channel.pckl'
            filelist=glob.glob(s_files)
            for trainfile in filelist:  #this list is 150 files long, ie 3pitch*50 samples per 1 class
                src=trainfile.split('\\')[-1].split('_')[0]
                bgs=''.join(i for i in src if i.isdigit())
                if int(bgs) in validationIDs: #then is a validation file
                     vf_list.append(trainfile)
                else:
                    tf_list.append(trainfile)
                
    print('training list is '+str(len(tf_list))+' long')
    print('validation list is '+str(len(vf_list))+' long')
    

    #now go through filelist (tf_list) to create X_train; should be 2100,128,1292,2
    X_train=np.zeros((len(tf_list),1292,128,2))
    Y_train=np.zeros((len(tf_list),1292,nb_classes))
    
    for i,tfile in enumerate(tf_list):
        fv,l=getStackFeatMat(tfile)
        X_train[i,:,:,:]=np.swapaxes(fv,0,1)
        Y_train[i,:]=l
    return X_train, Y_train, tf_list, vf_list

#X_train, Y_train, tf_list, vf_list = getTrainDataForFold(fold)

#print('ta da')
#print(X_train.shape)
#print(Y_train.shape)

#setup validation data
def getValDataForFold(vf_list):
#now go through validation (vf_list) to create X_train; should be 960,128,1292,2
    X_val=np.zeros((len(vf_list),1292,128,2))
    Y_val=np.zeros((len(vf_list),1292,nb_classes))
    for i,vfile in enumerate(vf_list):
        fv,l=getStackFeatMat(vfile)
        X_val[i,:,:,:]=np.swapaxes(fv,0,1)
        Y_val[i,:]=l
    return X_val, Y_val

#X_val,Y_val=getValDataForFold(vf_list)
#print('ta da')
#print(X_val.shape)
#print(Y_val.shape)

def getTestDataForFold(tef_list):
    print('testing list is '+str(len(tef_list))+' long')
    X_test=np.zeros((len(tef_list),1292,128,2))
    Y_test=np.zeros((len(tef_list),1292,nb_classes))    
    for i,tefile in enumerate(tef_list):
        fv,l=getStackFeatMat(tefile)
        X_test[i,:,:,:]=np.swapaxes(fv,0,1)
        Y_test[i,:]=l
    return X_test, Y_test

#src_files='stacked_for_classification\\test\\'+str(fold)+'\\*2channel.pckl'
#tef_list=glob.glob(src_files)
#X_test,Y_test=getTestDataForFold(tef_list)
#print(X_test.shape)
#print('test data sorted')


In [4]:
#scenes
batch_size =  32
nb_classes = 42
# input dimensions  
rows, cols , channels = 1292, 128,2    
#model params
nb_epoch = 50
pool_size = (3,3)                  # size of pooling area for max pooling
prob_drop_conv = 0.25                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer


###### Convolutional model
def compileCRNN(cols,rows,channels,nb_classes=1):
    model = Sequential()
    # conv1 layer
   #  model.add(Convolution1D(16, (3), padding='same', activation='relu', input_shape=(cols,channels)))
   
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu', input_shape=(rows,cols,channels)))
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(prob_drop_conv))

    # conv2 layer
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(Dropout(prob_drop_conv))

    # conv3 layer
    model.add(Conv2D(1292, (2, 2), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size, strides=(2), padding='same'))
    model.add(BatchNormalization())
   # model.add(Flatten())
    model.add(Dropout(prob_drop_conv))

    # lstm layer
    model.add(Reshape((1292,-1)))
    #model.add(Permute((2,1)))
    model.add(LSTM(1292,input_shape=(rows, cols),return_sequences=True))
  #  model.add(Reshape((256,-1)))

    # fc1 layer
    model.add(TimeDistributed(Dense(1292, activation='relu')))
    model.add(Dropout(prob_drop_hidden))
    
    # fc2 layer
    model.add(BatchNormalization())
    model.add(TimeDistributed(Dense(nb_classes, activation='sigmoid')))
    
    opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    #opt = RMSprop(lr=0.001, rho=0.9)
    #opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    model.summary()
    
    return model

model = compileCRNN(cols,rows,channels,nb_classes=nb_classes)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1292, 128, 256)    4864      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 646, 64, 256)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 646, 64, 256)      1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 646, 64, 256)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 646, 64, 512)      1180160   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 323, 32, 512)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 323, 32, 512)      0         
__________

In [5]:
# Train
#fold=1
def train_network(model, model_name, X_train, Y_train, X_val, Y_val, nb_epoch, validationsplit_size, batchsize, early_stoping_patience, output_folder):
    checkpointer = ModelCheckpoint(filepath=os.path.join(output_folder,model_name + '.hdf5'),save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=early_stoping_patience)
    tensorboard = TensorBoard(log_dir="logs\\{}".format(time()))
    Callbacks=[checkpointer,  tensorboard] #early_stopping,
  #  print(samples)
    steps=int(samples/batchsize)
    validationsteps=int(validationsplit_size/batchsize)
    history = model.fit(X_train, Y_train, epochs=nb_epoch, callbacks=Callbacks, batch_size=batch_size, validation_data=(X_val, Y_val), shuffle=True, verbose=1)
    return history,model

def buildModel(savemodelfilename, samples,model,X_train, Y_train,X_val, Y_val,fold ):
    valSplit_size = int(samples/4)
    early_stoping_patience=10
    history,model = train_network(model, '2D_models\\f'+str(fold)+'-best_model-strongFramesV2', X_train, Y_train, X_val, Y_val, nb_epoch, valSplit_size, batch_size, early_stoping_patience,'.')
    model.save_weights(savemodelfilename)
    return model, history

#testruncode - all fold run in cell below
#savemodelfilename='2D_models\\f'+str(fold)+'_model.testsave'
#samples=(X_train.shape[0])
#print(X_train.shape)
#models[fold],histories[fold] = buildModel(savemodelfilename, samples,model,X_train_mfcc,Y_train_mfcc,X_val_mfcc, Y_val_mfcc,)
#models,histories= buildModel(savemodelfilename, samples,model,X_train,Y_train,X_val, Y_val)


In [7]:
from datetime import datetime
batch_size =  8
nb_classes = 42
rows, cols , channels = 1292, 128,2    
nb_epoch = 100
pool_size = (3,3)                  # size of pooling area for max pooling
prob_drop_conv = 0.25                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer
#fold=1

for fold in range(1,2):
    print(fold)
    #sort data
    X_train, Y_train, tf_list, vf_list = getTrainDataForFold(fold)
    print('data obtained')
    X_val,Y_val=getValDataForFold(vf_list)
    print('lists sorted')
    model = compileCRNN(cols,rows,channels,nb_classes=nb_classes)
    print('model compiled')
    savemodelfilename='2D_models\\f'+str(fold)+'_model-strongFramesV2.testsave'
    samples=(X_train.shape[0])
    print(datetime.now())
    models,histories= buildModel(savemodelfilename, samples,model,X_train,Y_train,X_val, Y_val,fold)
    print(datetime.now())
    print('model saved')

1
[7]
training list is 2100 long
validation list is 300 long
data obtained
lists sorted
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 1292, 128, 256)    4864      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 646, 64, 256)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 646, 64, 256)      1024      
_________________________________________________________________
dropout_9 (Dropout)          (None, 646, 64, 256)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 646, 64, 512)      1180160   
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 323, 32, 512)      0         
______________________________________________________

Epoch 34/100
2100/2100 [==============================] - 1146s 546ms/step - loss: 0.0189 - binary_accuracy: 0.9927 - val_loss: 0.2919 - val_binary_accuracy: 0.9522
Epoch 35/100
2100/2100 [==============================] - 1147s 546ms/step - loss: 0.0182 - binary_accuracy: 0.9930 - val_loss: 0.2771 - val_binary_accuracy: 0.9567
Epoch 36/100
2100/2100 [==============================] - 1148s 546ms/step - loss: 0.0385 - binary_accuracy: 0.9866 - val_loss: 0.2488 - val_binary_accuracy: 0.9516
Epoch 37/100
2100/2100 [==============================] - 1146s 546ms/step - loss: 0.0238 - binary_accuracy: 0.9910 - val_loss: 0.2469 - val_binary_accuracy: 0.9516
Epoch 38/100
2100/2100 [==============================] - 1147s 546ms/step - loss: 0.0197 - binary_accuracy: 0.9925 - val_loss: 0.2589 - val_binary_accuracy: 0.9525
Epoch 39/100
2100/2100 [==============================] - 1146s 546ms/step - loss: 0.0180 - binary_accuracy: 0.9932 - val_loss: 0.2901 - val_binary_accuracy: 0.9490
Epoch 40/1

2100/2100 [==============================] - 1147s 546ms/step - loss: 0.0046 - binary_accuracy: 0.9984 - val_loss: 0.4481 - val_binary_accuracy: 0.9496
Epoch 84/100
2100/2100 [==============================] - 1146s 546ms/step - loss: 0.0045 - binary_accuracy: 0.9985 - val_loss: 0.4430 - val_binary_accuracy: 0.9506
Epoch 85/100
2100/2100 [==============================] - 1147s 546ms/step - loss: 0.0046 - binary_accuracy: 0.9984 - val_loss: 0.4211 - val_binary_accuracy: 0.9515
Epoch 86/100
2100/2100 [==============================] - 1147s 546ms/step - loss: 0.0043 - binary_accuracy: 0.9986 - val_loss: 0.3854 - val_binary_accuracy: 0.9530
Epoch 87/100
2100/2100 [==============================] - 1146s 546ms/step - loss: 0.0047 - binary_accuracy: 0.9984 - val_loss: 0.4417 - val_binary_accuracy: 0.9462
Epoch 88/100
2100/2100 [==============================] - 1147s 546ms/step - loss: 0.0048 - binary_accuracy: 0.9984 - val_loss: 0.4386 - val_binary_accuracy: 0.9515
Epoch 89/100
2100/2100 

In [1]:
X_val,Y_val=getValDataForFold(vf_list)

NameError: name 'getValDataForFold' is not defined

In [ ]:
#proper ASC - majority voting prediction?

In [ ]:
#proper SED - on/off set detection? 
